# Emergence Lab - Parallel Multi-Seed Training on TPU

This notebook runs **5 seeds in parallel** on Google Colab TPU.

**Goal**: Run 30 seeds Field ON + 30 seeds Field OFF = 60 total runs for statistical significance.

## Quick Start

| Step | What to do | Time |
|------|------------|------|
| **0** | **Run TEST MODE first** (cell after setup) | ~2 min |
| 1 | If test passes, set `TEST_MODE = False`, then `BATCH_NUMBER = 0` | ~14 min |
| 2 | Change `BATCH_NUMBER = 1`, run again | ~14 min |
| ... | Continue batches 2-5 | ... |
| 7 | Set `FIELD_ENABLED = False`, `BATCH_NUMBER = 0` | ~14 min |
| ... | Continue batches 1-5 | ... |

**Total time**: ~3 hours for all 60 seeds (after test passes).

## Setup Instructions

1. Open this notebook in Google Colab
2. Runtime > Change runtime type > **TPU v5e** (or v6e)
3. Run setup cells (1-4)
4. **Run TEST MODE cell first** - verify it passes before full training
5. Then run full training cells

## Cell 1: Setup

Mount Google Drive, clone the repo, and install dependencies.

In [ ]:
# Mount Google Drive for checkpoint storage
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory
import os
CHECKPOINT_BASE = '/content/drive/MyDrive/emergence-lab'
os.makedirs(CHECKPOINT_BASE, exist_ok=True)
print(f"Checkpoint base: {CHECKPOINT_BASE}")

In [ ]:
# Clone the repo if not already present
REPO_DIR = '/content/emergence-lab'

# ⚠️ CHANGE THIS to your GitHub username
GITHUB_USERNAME = "imashishkh"

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/{GITHUB_USERNAME}/emergence-lab.git {REPO_DIR}
else:
    print(f"Repo already exists at {REPO_DIR}")
    # Pull latest changes
    !cd {REPO_DIR} && git pull

# Change to repo directory
os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies
!pip install -e ".[dev]" -q

# Verify JAX sees TPU
import jax
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"Device count: {jax.device_count()}")

# Check if TPU is available
if 'tpu' in str(jax.devices()[0]).lower():
    print("TPU detected!")
else:
    print("WARNING: TPU not detected. Training will be slower on GPU/CPU.")

## ⚠️ TEST MODE - Run This First!

Before running the full experiment, do a quick test to verify everything works.
This uses ~1% of compute and catches errors early.

In [ ]:
# =============================================================================
# 🧪 TEST RUN - Verify parallel training works before full experiment
# =============================================================================
# This runs 2 seeds for 100K steps (~1-2 minutes) to catch errors early.
# If this succeeds, the full experiment should work.

TEST_MODE = True  # Set to False to skip test and go straight to full training

if TEST_MODE:
    print("="*60)
    print("🧪 TEST MODE: Running quick verification...")
    print("="*60)
    
    from src.configs import Config
    from src.training.parallel_train import ParallelTrainer
    import time
    
    # Minimal test config
    test_config = Config()
    test_config.env.num_food = 20
    test_config.evolution.enabled = True
    test_config.evolution.max_agents = 32
    test_config.evolution.starting_energy = 200
    test_config.evolution.food_energy = 100
    test_config.evolution.reproduce_threshold = 120
    test_config.evolution.reproduce_cost = 50
    test_config.train.num_envs = 32
    test_config.train.num_steps = 128
    test_config.log.wandb = False
    
    # Test parameters: 2 seeds, 100K steps
    TEST_STEPS = 100_000
    TEST_SEEDS = 2
    TEST_CHECKPOINT_DIR = f'{CHECKPOINT_BASE}/test_run'
    
    steps_per_iter = 32 * 128 * 32  # num_envs * num_steps * max_agents
    test_iterations = TEST_STEPS // steps_per_iter
    
    print(f"Test config:")
    print(f"  Seeds: {TEST_SEEDS}")
    print(f"  Steps: {TEST_STEPS:,}")
    print(f"  Iterations: {test_iterations}")
    print(f"  Checkpoint dir: {TEST_CHECKPOINT_DIR}")
    print()
    
    # Run test
    try:
        test_trainer = ParallelTrainer(
            config=test_config,
            num_seeds=TEST_SEEDS,
            seed_ids=[100, 101],  # Use different IDs to avoid conflicts
            checkpoint_dir=TEST_CHECKPOINT_DIR,
            master_seed=9999,
        )
        
        t0 = time.time()
        test_metrics = test_trainer.train(
            num_iterations=test_iterations,
            checkpoint_interval_minutes=60,  # Won't trigger in short test
            resume=False,
            print_interval=5,
        )
        elapsed = time.time() - t0
        
        print()
        print("="*60)
        print("✅ TEST PASSED!")
        print("="*60)
        print(f"Time: {elapsed:.1f}s")
        print(f"Final reward: {test_metrics.get('mean_reward', ['N/A'])}")
        print()
        print("The parallel training code works. You can now run the full experiment.")
        print("Set TEST_MODE = False in this cell and run the cells below.")
        print("="*60)
        
    except Exception as e:
        print()
        print("="*60)
        print("❌ TEST FAILED!")
        print("="*60)
        print(f"Error: {e}")
        print()
        print("DO NOT proceed with full training until this is fixed.")
        print("Share the error message to debug.")
        print("="*60)
        raise e

else:
    print("TEST_MODE = False, skipping test.")
    print("Proceeding to full training configuration...")

## Cell 2: Configuration

Configure which seeds to run and training parameters.

In [ ]:
# =============================================================================
# CONFIGURATION - Edit these values as needed
# =============================================================================

# Which batch of seeds to run (0-5 for 30 total seeds)
# Batch 0: seeds 0-4
# Batch 1: seeds 5-9
# Batch 2: seeds 10-14
# Batch 3: seeds 15-19
# Batch 4: seeds 20-24
# Batch 5: seeds 25-29
BATCH_NUMBER = 0  # Change this for each Colab session

# Seeds per batch (5 fits comfortably in 16GB TPU memory)
SEEDS_PER_BATCH = 5

# Compute seed IDs for this batch
SEED_IDS = list(range(BATCH_NUMBER * SEEDS_PER_BATCH, (BATCH_NUMBER + 1) * SEEDS_PER_BATCH))

# Training parameters
TOTAL_STEPS = 10_000_000  # Total environment steps per seed
NUM_ENVS = 32  # Parallel environments per seed
NUM_STEPS = 128  # Steps per rollout

# Field ablation setting
# Set to True for normal training, False for ablation (no field)
FIELD_ENABLED = True

# Checkpoint directory - SEPARATE for field_on vs field_off
CONDITION_NAME = "field_on" if FIELD_ENABLED else "field_off"
CHECKPOINT_DIR = f'{CHECKPOINT_BASE}/{CONDITION_NAME}/batch_{BATCH_NUMBER}'

# Checkpoint interval in minutes
CHECKPOINT_INTERVAL_MINUTES = 30

# Resume from existing checkpoints?
RESUME = True

# =============================================================================
# Print configuration summary
# =============================================================================
print("="*60)
print("Configuration Summary")
print("="*60)
print(f"Condition: {CONDITION_NAME}")
print(f"Batch number: {BATCH_NUMBER}")
print(f"Seed IDs: {SEED_IDS}")
print(f"Total steps per seed: {TOTAL_STEPS:,}")
print(f"Checkpoint directory: {CHECKPOINT_DIR}")
print(f"Checkpoint interval: {CHECKPOINT_INTERVAL_MINUTES} minutes")
print(f"Field enabled: {FIELD_ENABLED}")
print(f"Resume from checkpoints: {RESUME}")
print("="*60)

In [ ]:
# Build configuration
from src.configs import Config, EnvConfig, EvolutionConfig, FieldConfig

# Start with default config
config = Config()

# =============================================================================
# SURVIVAL-FRIENDLY PARAMETERS (proven to work in overnight runs)
# Without these, agents die off and training fails!
# =============================================================================
config.env.num_food = 20              # More food (default 10)
config.evolution.enabled = True
config.evolution.max_agents = 32      # Population cap
config.evolution.starting_energy = 200  # More starting energy (default 100)
config.evolution.food_energy = 100    # More energy per food (default 50)
config.evolution.energy_per_step = 1  # Energy drain per step
config.evolution.reproduce_threshold = 120  # Easier to reproduce (default 150)
config.evolution.reproduce_cost = 50  # Cheaper reproduction (default 80)
config.evolution.mutation_std = 0.01  # Standard mutation rate

# Training parameters
config.train.total_steps = TOTAL_STEPS
config.train.num_envs = NUM_ENVS
config.train.num_steps = NUM_STEPS
config.train.seed = 42  # Master seed (individual seeds are derived)

# Field ablation (if disabled, zero out the field)
if not FIELD_ENABLED:
    # Zero out field by setting decay to 1.0 (field becomes zero before observation)
    config.field.decay_rate = 1.0
    config.field.diffusion_rate = 0.0
    config.field.write_strength = 0.0

# Logging (disable W&B for parallel training)
config.log.wandb = False
config.log.save_interval = 0  # We handle checkpointing ourselves

print("Configuration loaded:")
print(f"  Grid size: {config.env.grid_size}")
print(f"  Num food: {config.env.num_food}")
print(f"  Max agents: {config.evolution.max_agents}")
print(f"  Starting energy: {config.evolution.starting_energy}")
print(f"  Food energy: {config.evolution.food_energy}")
print(f"  Reproduce threshold: {config.evolution.reproduce_threshold}")
print(f"  Field channels: {config.field.num_channels}")
print(f"  Field enabled: {FIELD_ENABLED}")
print(f"  Evolution enabled: {config.evolution.enabled}")

## Cell 3: Training

Run parallel training with automatic checkpointing.

In [ ]:
from src.training.parallel_train import ParallelTrainer

# Calculate number of iterations
steps_per_iter = NUM_ENVS * NUM_STEPS * config.evolution.max_agents
num_iterations = TOTAL_STEPS // steps_per_iter

print(f"Steps per iteration: {steps_per_iter:,}")
print(f"Number of iterations: {num_iterations:,}")
print()

# Create trainer
trainer = ParallelTrainer(
    config=config,
    num_seeds=SEEDS_PER_BATCH,
    seed_ids=SEED_IDS,
    checkpoint_dir=CHECKPOINT_DIR,
    master_seed=42 + BATCH_NUMBER * 1000,  # Different master seed per batch
)

# Run training
metrics = trainer.train(
    num_iterations=num_iterations,
    checkpoint_interval_minutes=CHECKPOINT_INTERVAL_MINUTES,
    resume=RESUME,
    print_interval=50,
)

In [ ]:
# Print final metrics summary
import numpy as np

print("\n" + "="*60)
print("Final Metrics Summary")
print("="*60)

for metric_name, values in metrics.items():
    mean = np.mean(values)
    std = np.std(values)
    print(f"{metric_name}:")
    print(f"  Mean: {mean:.4f} +/- {std:.4f}")
    print(f"  Per-seed: {[f'{v:.4f}' for v in values]}")
    print()

## Cell 4: Verify Checkpoints

List saved checkpoints and print summary.

In [ ]:
import glob
import pickle

print("Saved Checkpoints:")
print("="*60)

for seed_id in SEED_IDS:
    seed_dir = os.path.join(CHECKPOINT_DIR, f"seed_{seed_id}")
    checkpoints = sorted(glob.glob(os.path.join(seed_dir, "step_*.pkl")))
    
    print(f"\nSeed {seed_id}:")
    if checkpoints:
        for ckpt in checkpoints[-3:]:  # Show last 3
            size_mb = os.path.getsize(ckpt) / (1024 * 1024)
            print(f"  {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
        
        # Load latest and show step
        latest = os.path.join(seed_dir, "latest.pkl")
        if os.path.exists(latest):
            with open(latest, "rb") as f:
                data = pickle.load(f)
            print(f"  Latest step: {data.get('step', 'unknown')}")
    else:
        print("  No checkpoints found")

print("\n" + "="*60)
print("Checkpoint verification complete.")

In [ ]:
# Summary of all batches (run this to see overall progress)
print("Overall Progress Across All Conditions:")
print("="*60)

for condition in ["field_on", "field_off"]:
    print(f"\n{condition.upper()}:")
    print("-"*40)
    
    condition_complete = 0
    condition_expected = 30
    
    for batch in range(6):
        batch_dir = os.path.join(CHECKPOINT_BASE, condition, f"batch_{batch}")
        if os.path.exists(batch_dir):
            batch_seeds = list(range(batch * 5, (batch + 1) * 5))
            complete = 0
            for seed_id in batch_seeds:
                seed_dir = os.path.join(batch_dir, f"seed_{seed_id}")
                latest = os.path.join(seed_dir, "latest.pkl")
                if os.path.exists(latest):
                    try:
                        with open(latest, "rb") as f:
                            data = pickle.load(f)
                        step = data.get('step', 0)
                        if step >= TOTAL_STEPS * 0.99:  # 99% complete
                            complete += 1
                            condition_complete += 1
                    except:
                        pass
            print(f"  Batch {batch} (seeds {batch*5}-{batch*5+4}): {complete}/5 complete")
        else:
            print(f"  Batch {batch} (seeds {batch*5}-{batch*5+4}): Not started")
    
    print(f"  → {condition} total: {condition_complete}/{condition_expected} seeds")

print("\n" + "="*60)
print("Once both conditions have 30 seeds, run the report generator:")
print("python scripts/generate_multi_seed_report.py \\")
print(f"    --checkpoint-dir {CHECKPOINT_BASE}/field_on \\")
print(f"    --compare-dir {CHECKPOINT_BASE}/field_off")

## Next Steps

### Running the Full Experiment (60 seeds total)

**Phase 1: Field ON (30 seeds)**
1. Set `FIELD_ENABLED = True` and `BATCH_NUMBER = 0`, run notebook
2. Change `BATCH_NUMBER = 1`, run again
3. Repeat for batches 2, 3, 4, 5
4. Total time: ~6 × 14 min = ~1.5 hours

**Phase 2: Field OFF (30 seeds)**
1. Set `FIELD_ENABLED = False` and `BATCH_NUMBER = 0`, run notebook
2. Change `BATCH_NUMBER = 1`, run again
3. Repeat for batches 2, 3, 4, 5
4. Total time: ~6 × 14 min = ~1.5 hours

**Phase 3: Generate Report**
```bash
python scripts/generate_multi_seed_report.py \
    --checkpoint-dir /path/to/field_on \
    --compare-dir /path/to/field_off \
    --output-dir reports/multi_seed
```

### Directory Structure
```
emergence-lab/
├── field_on/
│   ├── batch_0/seed_0/, seed_1/, ... seed_4/
│   ├── batch_1/seed_5/, ... seed_9/
│   └── ...
└── field_off/
    ├── batch_0/seed_0/, ... seed_4/
    └── ...
```